Basic pybullet simulation of iterative movement based on franka_test.py IK code

In [1]:
!pip install opencv-python roboticstoolbox-python spatialmath-python
!pip install pybullet

import pybullet as p
import pybullet_data
import time
import numpy as np
import copy
import roboticstoolbox as rtb
from spatialmath import SE3
import cv2

# --- Connect to PyBullet ---
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# --- Load Environment & Robot ---
p.loadURDF("plane.urdf")
robot_id = p.loadURDF("franka_panda/panda.urdf", useFixedBase=True)

# Create a flat box (a thin rectangular box, e.g., 1m x 1m with a height of 0.01m)
flat_box_half_extents = [0.5, 0.5, 0.005]  # 1x1m box with a very thin height
flat_box_start_pos = [0, 0, 0.1]  # Position slightly above the ground
flat_box_collision_shape = p.createCollisionShape(p.GEOM_BOX, halfExtents=flat_box_half_extents)

# Load the PNG image
image_path = "Pickachu.png" # Replace with the path to your image
texture_id = p.loadTexture(image_path)

# Create a visual shape with the texture
box_visual_shape = p.createVisualShape(p.GEOM_BOX, halfExtents=[0.5, 0.5, 0.5], textureUniqueId=texture_id)

# Create a physics body and attach the visual shape
box_body = p.createMultiBody(1, box_visual_shape, p.createCollisionShape(p.GEOM_BOX, halfExtents=[0.5, 0.5, 0.5]))

# Set the initial position of the box
p.resetBasePositionAndOrientation(box_body, [0, 0, 1], [0, 0, 0, 1])


# --- Load Panda Model from Robotics Toolbox ---
panda = rtb.models.Panda()  # Robotics Toolbox Panda Model

# --- End-Effector Link Index ---
ee_link = 11  # Panda's end-effector

# --- Function to Get Current Joint Positions ---
def get_current_joint_positions(robot_id):
    return np.array([p.getJointState(robot_id, i)[0] for i in range(7)])

# --- Move Robot to Desired Pose Using `ikine_LM` ---
def move_to_pose(robot_id, panda, target_pose):
    """
    Moves the Panda robot smoothly to a target end-effector pose using `ikine_LM`.
    :param robot_id: PyBullet Panda robot ID
    :param panda: Panda model from Robotics Toolbox
    :param target_pose: SE3 desired pose of the end effector
    """
    # Get current joint positions
    current_joint_positions = get_current_joint_positions(robot_id)

    # Solve inverse kinematics using ikine_LM
    sol = panda.ikine_LM(target_pose, q0=current_joint_positions)

    if sol.success:
        print("IK Solution Found!")
        joint_solutions = sol.q
        print("Target Joint Angles:", joint_solutions)

        # Compute number of steps based on joint movement distance
        joint_distance = np.linalg.norm(joint_solutions - current_joint_positions)
        num_steps = int(np.clip(joint_distance * 100, 10, 1000))

        # Move smoothly to the new pose
        for alpha in np.linspace(0, 1, num_steps):
            intermediate_pos = (1 - alpha) * current_joint_positions + alpha * joint_solutions
            for i in range(7):
                p.setJointMotorControl2(robot_id, i, p.POSITION_CONTROL, targetPosition=intermediate_pos[i])
            p.stepSimulation()
            
            # --- Show Camera Feed during movement ---
            img = get_camera_image(robot_id)
            cv2.imshow("RealSense Camera", img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            time.sleep(1/240)

    else:
        print("Failed to find IK solution!")

# --- Attach a Simulated RealSense Camera to the EE ---
def get_camera_image(robot_id):
    """
    Captures an RGB-D image from a simulated camera mounted on the Panda's EE.
    """
    # --- Camera Parameters ---
    width, height = 640, 480
    fov = 60  # Field of view
    aspect = width / height
    near, far = 0.01, 2.0  # Near and far clipping planes

    # --- Get End-Effector Pose ---
    ee_state = p.getLinkState(robot_id, ee_link)
    ee_pos, ee_orient = ee_state[0], ee_state[1]

    # --- Convert Quaternion to Rotation Matrix ---
    rot_matrix = p.getMatrixFromQuaternion(ee_orient)
    rot_matrix = np.array(rot_matrix).reshape(3, 3)

    # --- Define Camera View (RealSense facing downward) ---
    forward_vector = np.dot(rot_matrix, [0, 0, 1])  # Z-axis in EE frame
    up_vector = np.dot(rot_matrix, [0, -1, 0])  # Y-axis in EE frame
    camera_pos = np.array(ee_pos) + forward_vector * 0.1  # Offset by 10 cm

    # --- Compute Look-at Target ---
    look_at = camera_pos + forward_vector

    # --- Compute View & Projection Matrices ---
    view_matrix = p.computeViewMatrix(camera_pos, look_at, up_vector)
    proj_matrix = p.computeProjectionMatrixFOV(fov, aspect, near, far)

    # --- Get Image from PyBullet ---
    img_arr = p.getCameraImage(width, height, view_matrix, proj_matrix)
    rgb_image = np.array(img_arr[2]).reshape(height, width, 4)[:, :, :3]

    # --- Convert RGB to OpenCV format ---
    rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

    return rgb_image

# --- Move to an Initial Pose and Show Camera Feed During Movement ---
start_pose = SE3(0.5, 0, 0.5) * SE3.RPY(-3.13, 0.097, 0.035)
move_to_pose(robot_id, panda, start_pose)

# --- Cleanup ---
cv2.destroyAllWindows()
p.disconnect()


pybullet build time: Apr  1 2025 12:02:39


Version = 4.1 Metal - 89.3
Vendor = Apple
Renderer = Apple M1 Pro


2025-04-01 21:18:18.901 python[40579:8433097] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-01 21:18:18.901 python[40579:8433097] +[IMKInputSession subclass]: chose IMKInputSession_Modern


b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


TypeError: 'textureUniqueId' is an invalid keyword argument for this function

: 

Texture issues with png.

In [1]:
!pip install opencv-python roboticstoolbox-python spatialmath-python
!pip install pybullet

import pybullet as p
import pybullet_data
import time
import numpy as np
import cv2
import roboticstoolbox as rtb
from spatialmath import SE3

# --- Connect to PyBullet ---
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# --- Load Environment & Robot ---
p.loadURDF("plane.urdf")  # Load the ground
robot_id = p.loadURDF("franka_panda/panda.urdf", useFixedBase=True)

# --- Load Panda Model from Robotics Toolbox ---
panda = rtb.models.Panda()

# --- End-Effector Link Index ---
ee_link = 11

# --- Create a Small Box for the Image Instead of a Plane ---
image_size = 0.3  # Adjust this for desired image area
image_position = [0.5, 0, 0.01]  # Slightly above the ground

# Create a small box instead of using a plane
visual_shape_id = p.createVisualShape(
    shapeType=p.GEOM_BOX,
    halfExtents=[image_size / 2, image_size / 2, 0.15],  # Thin box
)

collision_shape_id = p.createCollisionShape(
    shapeType=p.GEOM_BOX,
    halfExtents=[image_size / 2, image_size / 2, 0.001],  # Same as visual
)

image_plane_id = p.createMultiBody(
    baseMass=0,  # Static object
    baseCollisionShapeIndex=collision_shape_id,
    baseVisualShapeIndex=visual_shape_id,
    basePosition=image_position
)


# --- Load and Apply a Texture to the Small Box ---
image_path = "/Users/ericliang/Downloads/2025-03-31-160142.png"  # Replace with your image path
texture_id = p.loadTexture(image_path)
p.changeVisualShape(image_plane_id, -1, textureUniqueId=texture_id)



# --- Function to Get Current Joint Positions ---
def get_current_joint_positions(robot_id):
    return np.array([p.getJointState(robot_id, i)[0] for i in range(7)])

# --- Move Robot to Desired Pose Using `ikine_LM` ---
def move_to_pose(robot_id, panda, target_pose):
    """
    Moves the Panda robot smoothly to a target end-effector pose using `ikine_LM`.
    """
    current_joint_positions = get_current_joint_positions(robot_id)
    sol = panda.ikine_LM(target_pose, q0=current_joint_positions)

    if sol.success:
        joint_solutions = sol.q
        num_steps = int(np.clip(np.linalg.norm(joint_solutions - current_joint_positions) * 100, 10, 1000))

        for alpha in np.linspace(0, 1, num_steps):
            intermediate_pos = (1 - alpha) * current_joint_positions + alpha * joint_solutions
            for i in range(7):
                p.setJointMotorControl2(robot_id, i, p.POSITION_CONTROL, targetPosition=intermediate_pos[i])
            p.stepSimulation()

            img = get_camera_image(robot_id)
            cv2.imshow("RealSense Camera", img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

            time.sleep(1/240)
    else:
        print("Failed to find IK solution!")

# --- Attach a Simulated RealSense Camera to the EE ---
def get_camera_image(robot_id):
    width, height = 640, 480
    fov = 60
    aspect = width / height
    near, far = 0.01, 2.0

    ee_state = p.getLinkState(robot_id, ee_link)
    ee_pos, ee_orient = ee_state[0], ee_state[1]

    rot_matrix = np.array(p.getMatrixFromQuaternion(ee_orient)).reshape(3, 3)
    forward_vector = np.dot(rot_matrix, [0, 0, 1])
    up_vector = np.dot(rot_matrix, [0, -1, 0])
    camera_pos = np.array(ee_pos) + forward_vector * 0.1
    look_at = camera_pos + forward_vector

    view_matrix = p.computeViewMatrix(camera_pos, look_at, up_vector)
    proj_matrix = p.computeProjectionMatrixFOV(fov, aspect, near, far)

    img_arr = p.getCameraImage(width, height, view_matrix, proj_matrix)
    rgb_image = np.array(img_arr[2]).reshape(height, width, 4)[:, :, :3]
    rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

    return rgb_image

# --- Move to an Initial Pose and Show Camera Feed During Movement ---
start_pose = SE3(0.5, 0, 0.5) * SE3.RPY(-3.13, 0.097, 0.035)
move_to_pose(robot_id, panda, start_pose)

# --- Cleanup ---
cv2.destroyAllWindows()
p.disconnect()


pybullet build time: Apr  1 2025 12:02:39


Version = 4.1 Metal - 89.3
Vendor = Apple
Renderer = Apple M1 Pro
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


2025-04-01 21:42:39.428 python[41035:8453668] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-01 21:42:39.428 python[41035:8453668] +[IMKInputSession subclass]: chose IMKInputSession_Modern


KeyboardInterrupt: 

: 